In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import cv2
import os
import shutil

from glob import glob
from pathlib import Path
!pip install wolta
!pip install ultralytics
os.environ['WANDB_MODE'] = 'disabled'
for dirname, _, _ in os.walk('/kaggle/input'):
    print(dirname)
p_paths = glob('/kaggle/input/*')
d_paths = []

for p_path in p_paths:
    d_paths.extend(glob('{}/*'.format(p_path)))
print(d_paths)
i_paths = []

for d_path in d_paths:
    i_paths.extend(glob('{}/*'.format(d_path)))

print(len(i_paths))
from wolta.visual_tools import get_extensions

get_extensions(i_paths)
from wolta.visual_tools import dataset_size_same

dataset_size_same(i_paths)
temp_img = cv2.imread(i_paths[0])
ratio = temp_img.shape[1] / temp_img.shape[0]

print('Width: {}'.format(temp_img.shape[1]))
print('Height: {}'.format(temp_img.shape[0]))
print('Ratio: {}'.format(ratio))
os.makedirs('/kaggle/working/raw')
for d_path in d_paths:
    current_dir = Path(d_path).name
    current_path = '/kaggle/working/raw/{}'.format(current_dir) 
    os.makedirs(current_path, exist_ok=True)

    i_paths = glob('{}/*'.format(d_path))

    for i_path in i_paths:
        shutil.copy(i_path, current_path)
        
from wolta.visual_tools import dir_split

dir_split('/kaggle/working/raw', '/kaggle/working/data', test_size=0.2, val_size=0.2)

In [ ]:

from tqdm import tqdm

def add_gaussian_noise(image, mean=0, std=25):
    """Add Gaussian noise to an image."""
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image

def process_and_save_images(input_path, output_path):
    """Process images, add noise, and save to the output path."""
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for img_name in tqdm(os.listdir(input_path)):
        img_path = os.path.join(input_path, img_name)
        if os.path.isfile(img_path):
            # Read image
            image = cv2.imread(img_path)
            if image is None:
                continue
            # Add Gaussian noise
            noisy_image = add_gaussian_noise(image)
            # Save the noisy image to the output folder
            output_img_path = os.path.join(output_path, img_name)
            cv2.imwrite(output_img_path, noisy_image)
base_dir = "/kaggle/working/data"  # Replace with the root path of your dataset
subsets = [ "test"]
categories = ["REAL", "FAKE"]

for subset in subsets:
    real_path = os.path.join(base_dir, subset, "REAL")
    fake_path = os.path.join(base_dir, subset, "FAKE")

    print(f"Processing {subset}/REAL...")
    process_and_save_images(real_path, fake_path)


In [ ]:
from wolta.visual_tools import cls_img_counter

cls_img_counter('/kaggle/working/data')

from ultralytics import YOLO

model = YOLO(model='yolo11x-cls.pt')
results = model.train(data='/kaggle/working/data', epochs=5, imgsz=32, verbose= True)
test_results = model.val(data='/kaggle/working/data', imgsz=32, split="test")
# Save the trained model
model.save('trained_yolo_model.pt')